In [1]:
import sys
sys.path.append('../')

In [2]:
from tank import Tank, Hole, create_holes
import ipywidgets as widgets
from ipycanvas import canvas
from IPython.display import display, Markdown, Latex
import time
from demo import *

In [3]:
c = canvas.Canvas(width=600, height=200)
m = Tank(create_holes(25, 2), 0.06, c=c)
demo = Demo(m.params, m)

In [ ]:
demo.show()
c